In [1]:
from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:

dataframe = spark.createDataFrame(
    [
        (1, "2020-01-01", 1, 1),
        (2, "2020-01-01", 2, 1),
        (3, "2020-01-02", 1, 1),
        (2, "2020-01-02", 1, 1)
    ],
    ("id", "ts", "feature", "h3")
).withColumn("ts", F.col("ts").cast("timestamp"))

In [3]:


w=Window().partitionBy("id","h3").orderBy("ts")
w1=Window().partitionBy("id","h3").orderBy("ts").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
dataframe=dataframe.withColumn('last', F.row_number().over(w)).withColumn('min_last', F.max("last").over(w1)).filter("last=min_last").drop("last","min_last")

In [4]:
aggregated = dataframe.groupby("h3",
  F.window(
    timeColumn="ts",
    windowDuration="3 days",
    slideDuration="1 day",
  )
).agg(
  F.sum("feature")
)
aggregated.show(truncate=False)

+---+------------------------------------------+------------+
h3 |window |sum(feature)|
+---+------------------------------------------+------------+
1 |[2020-01-01 00:00:00, 2020-01-04 00:00:00]|3 |
1 |[2019-12-31 00:00:00, 2020-01-03 00:00:00]|3 |
1 |[2019-12-30 00:00:00, 2020-01-02 00:00:00]|1 |
1 |[2020-01-02 00:00:00, 2020-01-05 00:00:00]|2 |
+---+------------------------------------------+------------+

In [5]:
aggregated1 = dataframe.groupby("h3",
  F.window(
    timeColumn="ts",
    windowDuration="3 days",
    slideDuration="1 day",
  )
).agg(
  F.map_from_entries(
    F.collect_list(
      F.struct("id","feature")
    )
  ).alias("id_feature")
)
aggregated1.show(truncate=False)

+---+------------------------------------------+------------------------+
h3 |window |id_feature |
+---+------------------------------------------+------------------------+
1 |[2020-01-01 00:00:00, 2020-01-04 00:00:00]|[1 -> 1, 2 -> 1, 3 -> 1]|
1 |[2019-12-31 00:00:00, 2020-01-03 00:00:00]|[1 -> 1, 2 -> 1, 3 -> 1]|
1 |[2019-12-30 00:00:00, 2020-01-02 00:00:00]|[1 -> 1] |
1 |[2020-01-02 00:00:00, 2020-01-05 00:00:00]|[2 -> 1, 3 -> 1] |
+---+------------------------------------------+------------------------+

In [6]:
display(aggregated1)

h3,window,id_feature
1,"List(2020-01-01T00:00:00.000+0000, 2020-01-04T00:00:00.000+0000)","Map(1 -> 1, 2 -> 2, 3 -> 1)"
1,"List(2019-12-31T00:00:00.000+0000, 2020-01-03T00:00:00.000+0000)","Map(1 -> 1, 2 -> 2, 3 -> 1)"
1,"List(2019-12-30T00:00:00.000+0000, 2020-01-02T00:00:00.000+0000)","Map(1 -> 1, 2 -> 2)"
1,"List(2020-01-02T00:00:00.000+0000, 2020-01-05T00:00:00.000+0000)",Map(3 -> 1)


In [7]:
from pyspark.sql import functions as F

In [8]:
list=[['0 days 00:00:00.0'],
['0 days 00:30:00.0'],
['0 days 01:00:00.0'],
['0 days 01:30:00.0']]

df=spark.createDataFrame(list,['time'])
df.show()

+-----------------+
 time|
+-----------------+
0 days 00:00:00.0|
0 days 00:30:00.0|
0 days 01:00:00.0|
0 days 01:30:00.0|
+-----------------+

In [10]:
df.withColumn("Time_timestamp", F.unix_timestamp(F.substring("time",7,9),"HH:mm:ss")).show()

+-----------------+--------------+
 time|Time_timestamp|
+-----------------+--------------+
0 days 00:00:00.0| 0|
0 days 00:30:00.0| 1800|
0 days 01:00:00.0| 3600|
0 days 01:30:00.0| 5400|
+-----------------+--------------+

In [11]:
df.withColumn("Time_timestamp", F.to_timestamp(F.substring("time",7,9),"HH:mm:ss")).show()

+-----------------+-------------------+
 time| Time_timestamp|
+-----------------+-------------------+
0 days 00:00:00.0|1970-01-01 00:00:00|
0 days 00:30:00.0|1970-01-01 00:30:00|
0 days 01:00:00.0|1970-01-01 01:00:00|
0 days 01:30:00.0|1970-01-01 01:30:00|
+-----------------+-------------------+

In [12]:
df.withColumn("Time_timestamp", F.substring("time",7,9)).show()

+-----------------+--------------+
 time|Time_timestamp|
+-----------------+--------------+
0 days 00:00:00.0| 00:00:00|
0 days 00:30:00.0| 00:30:00|
0 days 01:00:00.0| 01:00:00|
0 days 01:30:00.0| 01:30:00|
+-----------------+--------------+